---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [161]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [162]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [194]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    with open("university_towns.txt") as f:
        u_towns = f.readlines()
    
    u_towns = [x.rstrip() for x in u_towns]
    u_towns_brackets = list()
    
    for x in u_towns:
        if x[-6:] == '[edit]':
            state = x[:-6]
        elif '(' in x:
            town = x[:x.index('(') - 1]
            u_towns_brackets.append([state, town])
        else:
            town = x
            u_towns_brackets.append([state, town])

    u_towns = pd.DataFrame(u_towns_brackets, columns=['State','RegionName'])


    return u_towns

get_list_of_university_towns()



Alabama[edit]
Auburn (Auburn University)[1]
Florence (University of North Alabama)
Jacksonville (Jacksonville State University)[2]
Livingston (University of West Alabama)[2]
Montevallo (University of Montevallo)[2]
Troy (Troy University)[2]
Tuscaloosa (University of Alabama, Stillman College, Shelton State)[3][4]
Tuskegee (Tuskegee University)[5]
Alaska[edit]
Fairbanks (University of Alaska Fairbanks)[2]
Arizona[edit]
Flagstaff (Northern Arizona University)[6]
Tempe (Arizona State University)
Tucson (University of Arizona)
Arkansas[edit]
Arkadelphia (Henderson State University, Ouachita Baptist University)[2]
Conway (Central Baptist College, Hendrix College, University of Central Arkansas)[2]
Fayetteville (University of Arkansas)[7]
Jonesboro (Arkansas State University)[8]
Magnolia (Southern Arkansas University)[2]
Monticello (University of Arkansas at Monticello)[2]
Russellville (Arkansas Tech University)[2]
Searcy (Harding University)[5]
California[edit]
Angwin (Pacific Union College

['Alabama[edit]',
 'Auburn (Auburn University)[1]',
 'Florence (University of North Alabama)',
 'Jacksonville (Jacksonville State University)[2]',
 'Livingston (University of West Alabama)[2]',
 'Montevallo (University of Montevallo)[2]',
 'Troy (Troy University)[2]',
 'Tuscaloosa (University of Alabama, Stillman College, Shelton State)[3][4]',
 'Tuskegee (Tuskegee University)[5]',
 'Alaska[edit]',
 'Fairbanks (University of Alaska Fairbanks)[2]',
 'Arizona[edit]',
 'Flagstaff (Northern Arizona University)[6]',
 'Tempe (Arizona State University)',
 'Tucson (University of Arizona)',
 'Arkansas[edit]',
 'Arkadelphia (Henderson State University, Ouachita Baptist University)[2]',
 'Conway (Central Baptist College, Hendrix College, University of Central Arkansas)[2]',
 'Fayetteville (University of Arkansas)[7]',
 'Jonesboro (Arkansas State University)[8]',
 'Magnolia (Southern Arkansas University)[2]',
 'Monticello (University of Arkansas at Monticello)[2]',
 'Russellville (Arkansas Tech Un

In [171]:
gdplev = pd.read_excel('gdplev.xls',header=219,names = ['Time', 'none','GDP Chained','none'])
gdplev = gdplev[['Time', 'GDP Chained']]
gdplev.set_index('Time', inplace=True)
gdplev = gdplev['GDP Chained']

def start_index():
    val = None
    for i in range(0, len(gdplev)-2):
        if gdplev[i+1] < gdplev[i] and gdplev[i+2] < gdplev[i+1]:
            val = i+1
            break
            
    return val
    
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    
    return gdplev.index[start_index()]

get_recession_start()

'2008q3'

In [165]:
def end_index():
    val_end = None
    start = start_index()
    for i in range(start, len(gdplev)-2):
        if gdplev[i+1] > gdplev[i] and gdplev[i+2] > gdplev[i+1]:
            val_end = i+2
            break
            
    return val_end
        
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    val = start_index()
    val_end = end_index()

    return gdplev.index[val_end]

get_recession_end()

'2009q4'

In [166]:
import sys
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    start = start_index()
    end = end_index()
    val = sys.maxsize
    for i in range(start, end):
        if gdplev[i] < val:
            minumum = i
            val = gdplev[i]
        
    return gdplev.index[minumum]

get_recession_bottom()

'2009q2'

In [191]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df= pd.read_csv("City_Zhvi_AllHomes.csv")
    df['State'].replace(states, inplace= True)
    df= df.set_index(["State","RegionName"])
    
    df = df.iloc[:,49:250]    
    def cutter(col):
        if col.endswith(("03", "02", "01")):
            return col[:4] + "q1"
        if col.endswith(("06", "05", "04")):
            return col[:4] + "q2"
        if col.endswith(("09", "08", "07")):
            return col[:4] + "q3"
        
        return col[:4] + "q4"
        
    home = df.groupby(cutter, axis = 1).mean()
    home = home.sort_index()
    return home

convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State     RegionName                                                     
Alabama   Adamsville        69033.333333   69166.666667   69800.000000   
          Alabaster        122133.333333  123066.666667  123166.666667   
          Albertville       73966.666667   72600.000000   72833.333333   
          Arab              83766.666667   81566.666667   81333.333333   
          Ardmore                    NaN            NaN            NaN   
          Axis                       NaN            NaN            NaN   
          Baileyton                  NaN            NaN            NaN   
          Bay Minette       81700.000000   78533.333333   79133.333333   
          Bayou La Batre    44066.666667   44500.000000   44266.666667   
          Bessemer                   NaN            NaN            NaN   
          Birmingham        54033.333333   54400.000000   54966.666667   
          Boaz              70866.666667   70266.666667   70300.000000   
          Brent             92933.333333   94333.333333   96166.666667   
          Brighton                   NaN            NaN            NaN   
          Brookwood         92566.666667   95100.000000   98866.666667   
          Buhl              90800.000000   94600.000000   96500.000000   
          Calera           108933.333333  110366.666667  108000.000000   
          Center Point      80966.666667   81233.333333   81500.000000   
          Centreville       95300.000000   96566.666667   98000.000000   
          Chalkville        94100.000000   94433.333333   94433.333333   
          Chancellor                 NaN            NaN            NaN   
          Chelsea          162066.666667  167033.333333  166900.000000   
          Chickasaw         51200.000000   53666.666667   54933.333333   
          Chunchula         80266.666667   81766.666667   82200.000000   
          Citronelle        64833.333333   66633.333333   68066.666667   
          Clay             120900.000000  122266.666667  123966.666667   
          Coden             62600.000000   64800.000000   66866.666667   
          Coker            118100.000000  120766.666667  118166.666667   
          Concord           78600.000000   78700.000000   80133.333333   
          Cottondale       100833.333333  102633.333333  104766.666667   
...                                  ...            ...            ...   
Wisconsin Vernon           183200.000000  178200.000000  174300.000000   
          Vienna           178033.333333  181533.333333  182433.333333   
          Vinland          119800.000000  126766.666667  134933.333333   
          Wales                      NaN            NaN            NaN   
          Waterford        121200.000000  119433.333333  120200.000000   
          Waukesha         141266.666667  138433.333333  136733.333333   
          Waunakee         187400.000000  191433.333333  192666.666667   
          Waupun            84000.000000   84000.000000   84400.000000   
          Wausau            69566.666667   69466.666667   71033.333333   
          Wayne                      NaN            NaN            NaN   
          West Allis        80933.333333   81933.333333   84066.666667   
          Weston            80166.666667   82700.000000   84166.666667   
          Wheatland        129800.000000  132133.333333  134933.333333   
          Whitelaw          96033.333333  101433.333333  108033.333333   
          Williams Bay     122900.000000  115300.000000  110766.666667   
          Wilson           129033.333333  129366.666667  132433.333333   
          Wilson                     NaN            NaN            NaN   
          Wind Lake        124366.666667  123666.666667  124133.333333   
          Wind Point       149233.333333  145266.666667  140866.666667   
          Wisconsin Dells  100000.000000  103400.000000  105000.000000   
          Wolf River        95166.666667   99333.333333  103800.000000   
          Woodruff                   N

In [192]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    start = get_recession_start()
    bottom = get_recession_bottom()
    u_town = get_list_of_university_towns()
    home = convert_housing_data_to_quarters()
    home.reset_index(inplace=True)
    
    home.State = home.State.replace(states)
    
    home_use = home[['State', 'RegionName', '2008q3', '2009q2']]
    
    home = home_use
    
    home['dif'] = home_use['2008q3'] - home_use['2009q2']
    
    diff_u = pd.merge(home, u_town, how='inner', on=['State', 'RegionName'])['dif'].dropna()
    diff_no_u_index = set(home.index) - set(diff_u.index)
    diff_no_u = home['dif'].loc[list(diff_no_u_index)].dropna()
    
    s, p = tuple(ttest_ind(diff_u, diff_no_u))

    different = p < 0.01
    better = "university town" if diff_u.mean() < diff_no_u.mean() else "non-university town"
    
    return different, p, better

run_ttest()

(True, 0.0052909052169765143, 'university town')